In [1]:
#Importing libraies
!pip install -q langchain langgraph langchain-core langchain-community transformers accelerate torch torchvision sentence-transformers evaluate gradio radon scikit-learn==1.5.1 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 96.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.6/212.6 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.


In [2]:
#Imporing libraries
from transformers import AutoTokenizer,AutoModelForCausalLM,pipeline
import torch
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_core.output_parsers import StrOutputParser
import gradio as gr
import re

2026-02-23 11:34:28.822512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771846468.999070      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771846469.048004      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771846469.470837      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771846469.470875      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771846469.470878      55 computation_placer.cc:177] computation placer alr

In [103]:
#Loading llm model and tokenizer 
MODEL_NAME = "microsoft/phi-3-mini-4k-instruct" #lightweight instruction tuned
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map='auto'
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    max_new_tokens = 500,
    temperature=0.2
)

Device set to use cuda:0


In [104]:
llm = HuggingFacePipeline(pipeline=pipe)

In [105]:
#Define Agents

#Product Manager Agent
pm_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a senior Product Manager. Break down this feature request into clear and concise technical tasks:"),
    ("user", "Feature request:\n{feature}\nDon't provide any code implementations or examples. Also, Don't hallucinate"),
    ("assistant", " ")
])

pm_chain = pm_prompt | llm | StrOutputParser()

In [106]:
#Architect Agent

architect_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a senior Software Architect. Design a technical architecture of the following technical tasks."),
    ("user", "Product specification:\n{pm_output}\n Provide a clear and concise technical architecture in a short pointwise structure for every technical task."),
    ("assistant", " ")
])

architect_chain = architect_prompt | llm | StrOutputParser()

In [158]:
#Developer Agent

dev_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Senior Python developer. Write clean production-ready code based on the following Architecture design."),
    ("user", "Architecture design:\n{architecture}\nProvide only modular python code and donot include the code pointwise."),
    ("assistant", " ")
])

dev_chain = dev_prompt | llm | StrOutputParser()

In [108]:
#Tester Agent

tester_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a QA engineer. Write unit tests for the following code."),
    ("user", "Code:\n{code}\nOnly accept python code as input and output unit test cases."),
    ("assistant", " ")
])

tester_chain = tester_prompt | llm | StrOutputParser()

In [109]:
#Reviewer Agent
review_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a senior code reviewer. Provide feedback and improvements of the following code"),
    ("user", "Code:\n{code}\nAccept python code as input and only output the short pointwise improvements or suggestions according to the code."),
    ("assistant", " ")
])

review_chain = review_prompt | llm | StrOutputParser()

In [23]:
#Function to clean the text and only output generated answer.
def clean_output(text: str) -> str:
    if "AI:" in text:
        text = text.split("AI:")[-1]
    return text

In [159]:
#Orchestrating the langgraph

#Define the state structure or memory to hold data.
class SprintState(TypedDict):
    feature: str
    pm_output: str
    architecture: str
    code: str
    tests: str
    review: str

#Define the node logic
def pm_node(state):
    res = pm_chain.invoke({"feature":state["feature"]})
    print(res)
    return {"pm_output": res}

def architect_node(state):
    res = architect_chain.invoke({"pm_output":clean_output(state["pm_output"])})
    print(res)
    return {"architecture":res}

def dev_node(state):
    res = dev_chain.invoke({"architecture":clean_output(state["architecture"])})
    print(res)
    return {"code":res}

def tester_node(state):
    res = tester_chain.invoke({"code":clean_output(state["code"])})
    print(res)
    return {"tests":res}

def review_node(state):
    res  = review_chain.invoke({"code":clean_output(state["code"])})
    print(res)
    return {"review": res}


In [160]:
#Intialize State Graph
graph = StateGraph(SprintState)

#Add nodes and edges
graph.add_node("pm",pm_node)
graph.add_node("architecture",architect_node)
graph.add_node("dev",dev_node)
graph.add_node("tester",tester_node)
graph.add_node("review",review_node)

graph.set_entry_point("pm")
graph.add_edge("pm","architecture")
graph.add_edge("architecture","dev")
graph.add_edge("dev","tester")
graph.add_edge("tester","review")
graph.add_edge("review",END)

#Compile 
app = graph.compile()

In [169]:
#Run the sprint

result = app.invoke({
    "feature": "Build a rate limiter middleware for a FastAPI app."
})

System: You are a senior Product Manager. Break down this feature request into clear and concise technical tasks:
Human: Feature request:
Build a rate limiter middleware for a FastAPI app.
Don't provide any code implementations or examples. Also, Don't hallucinate
AI:  1. Research and select a suitable rate limiting algorithm or library.
2. Integrate the chosen rate limiting library into the FastAPI application.
3. Configure the rate limiting settings (e.g., maximum requests per minute, time window).
4. Implement the rate limiter middleware in the FastAPI app.
5. Test the rate limiter middleware to ensure it works as expected.
6. Document the implementation and configuration of the rate limiter middleware.
7. Monitor and adjust the rate limiting settings as needed based on application usage and performance.


System: You are a senior Product Manager. Break down this feature request into clear and concise technical tasks:
Human: Feature request:
Build a rate limiter middleware for a Fas

In [175]:
#Remove python and triple quotes
def remove_quotes(text):
    python_removed = re.sub(r"```[a-zA-Z]*", "", text)
    quotes_removed = python_removed.replace("```", "")
    quotes_removed = python_removed.replace("`", "").strip()
    digits_detect = re.sub(r"[0-9]+\.","#",quotes_removed)
    
    return digits_detect.strip()

In [176]:
#extract code for radon
code_only = clean_output(result['code'])
code_only = remove_quotes(code_only)
print(code_only)

# Research and select a suitable rate limiting algorithm or library:


# Research and select a suitable rate limiting algorithm or library
# For this example, we'll use the "limits" library which provides a Token Bucket algorithm
# Install the library using pip
# pip install limits

import limits
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# Configure CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Define rate limiting settings
rate_limit = limits.RateLimit(
    limit=10,  # Maximum requests per minute
    period=60,  # Time window in seconds
    key_func=lambda request: request.client.host,  # Unique key per client IP
)

# Create a rate limiting instance
limiter = limits.RateLimiter(rate_limit)

# Implement the rate limiter middleware
class RateLimiterMiddleware:
    

In [177]:
#Evaulation metrics
#1. Code complexity (Cyclomatic Code Complexity)
import radon.complexity as rc
complexity = rc.cc_visit(code_only)
for block in complexity:
    print(f"{block.name}: {block.complexity}")
### Since the code complexity is in range of 1-5, then it is categorized as A (Simple;Low risk blocks)

test_rate_limiter: 2
RateLimiterMiddleware: 3
__call__: 2


In [178]:
#2. Test coverage: More the number of test, more test coverage
test_cases_only = clean_output(result["tests"])
test_cases = remove_quotes(test_cases_only)
count=0
if 'def test_' in test_cases:
    count+=1
else:
    pass
print("Valid Test Cases:", count)

Valid Test Cases: 1


In [179]:
#3. Semantic Similarity to feature
embedder = SentenceTransformer("all-MiniLM-L6-v2")
emb_feature = embedder.encode(result["feature"])
emb_code = embedder.encode(result["code"])

similarity = cosine_similarity([emb_feature], [emb_code])[0][0]
print("Feature-Code Alignment:", similarity)

Feature-Code Alignment: 0.75720924


In [180]:
#4. LLM Self-Reflection Score
evaluation_prompt = PromptTemplate(
    input_variables=["feature","code"],
    template = """
    Score the implementation from 1-10 for:
    -Correctness
    -Maintainability
    -Completeness

    Feature:
    {feature}

    Code:
    {code}

    """
)
evaluation_chain = evaluation_prompt | llm | StrOutputParser()

score = evaluation_chain.invoke({'feature':result['feature'],'code':result['code']})
print(score)


    Score the implementation from 1-10 for:
    -Correctness
    -Maintainability
    -Completeness

    Feature:
    Build a rate limiter middleware for a FastAPI app.

    Code:
    System: You are a Senior Python developer. Write clean production-ready code based on the following Architecture design.
Human: Architecture design:
  1. Research and select a suitable rate limiting algorithm or library:
   - Conduct research on various rate limiting algorithms and libraries available, such as token bucket, leaky bucket, fixed window, sliding window, etc.
   - Evaluate the pros and cons of each algorithm and library based on factors like performance, scalability, ease of implementation, and compatibility with FastAPI.
   - Choose a rate limiting algorithm or library that best fits the application's requirements and constraints.

2. Integrate the chosen rate limiting library into the FastAPI application:
   - Install the chosen rate limiting library using a package manager like pip.
   - 

In [153]:
def run_sprint(feature):
    return app.invoke({"feature": feature})

interface = gr.Interface(
    fn=run_sprint,
    inputs="text",
    outputs="json"
)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://71e66c22f0626e7db8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


System: You are a senior Product Manager. Break down this feature request into clear and concise technical tasks:
Human: Feature request:
Build a CI/CD pipeline to automate a pipeline.
Don't provide any code implementations or examples. Also, Don't hallucinate
AI:  1. Define the CI/CD pipeline architecture:
   - Identify the tools and services to be used (e.g., Jenkins, GitLab CI, GitHub Actions).
   - Determine the stages of the pipeline (e.g., build, test, deploy).

2. Set up the repository and version control:
   - Ensure the codebase is in a version control system (e.g., Git).
   - Create branches for development, testing, and production.

3. Configure the build process:
   - Set up build scripts or use build tools (e.g., Maven, Gradle, Docker).
   - Define build triggers (e.g., on push, pull request, or schedule).

4. Implement automated testing:
   - Integrate testing frameworks (e.g., JUnit, Selenium, PyTest).
   - Define test cases and test suites.
   - Configure test execution